In [159]:
# !pip install tqdm
import pandas as pd
import re
import numpy as np
import pickle
import scipy
import os
import string
from tqdm import tqdm as timer
import sklearn

In [71]:
movie_ids = {}
with open('movies_2.txt') as f:
    i = 0
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        movie_ids[line] = i
        
        i += 1
        if(i%100000==0):
            print(i)
#         line = line.decode('utf-8', 'ignore').encode('utf-8')
        

100000
200000
300000
400000


In [72]:
def remove_extra(s):
    return(re.sub(r'\([^\d]*\)|\{.*\}|\'|\"', "", s).strip())

In [73]:
def clean_movie_rating(file_name):
    movie_rating_map = {}
    i = 0
    with open(file_name, 'rb') as f:
        lines = f.readlines()
        for line in lines:
            i += 1
            if(i%100000==0):
                print(i)
            line = line.decode('utf-8', 'ignore').encode('utf-8')
            line = line.strip()
            line_split = line.decode().split('\t\t')
            movie_name = remove_extra(line_split[0])
            rating = float(remove_extra(line_split[1]))
            
            movie_rating_map[movie_name] = rating
            
    return movie_rating_map

In [74]:
movie_rating_map = clean_movie_rating('data/movie_rating.txt')

100000
200000
300000


In [76]:
def get_non_duplicated(artist_dataframe):
    artist_counts = artist_dataframe[['movies','director']].groupby('director',as_index=False).agg(['count'])
    artist_counts.columns = [''.join(col).strip() for col in artist_counts.columns.values]
    artist_counts2 = artist_counts.reset_index()
    print('Only',artist_counts2['director'].count(),' unique names')
    
    artist_counts2  = artist_counts2.loc[artist_counts2['moviescount'] >1]
    duplicate_artists = list(artist_counts2['director'].values)
    #print(len(duplicate_artists),' repeated names')
    duplicated_artist_df  = artist_dataframe.loc[artist_dataframe['director'].isin(duplicate_artists)]
    non_duplicated_artist_df  = artist_dataframe.loc[~artist_dataframe['director'].isin(duplicate_artists)]
    print(non_duplicated_artist_df['movies'].count(),' non repeated names')
    fixed_duplicated_artist_df = duplicated_artist_df.groupby('director').agg({'movies':'sum'})
    fixed_duplicated_artist_df['director'] = fixed_duplicated_artist_df['movies'].apply(lambda x: list(set(x)))
    print(fixed_duplicated_artist_df['director'].count(),' responsible for duplicates')
    non_duplicated_artist_df.append(fixed_duplicated_artist_df)
    return non_duplicated_artist_df


In [77]:
def get_director_data(file_name):
    artist_list = []
    artist_movies_list = []
    i = 0
    with open(file_name,'rb') as f:
        lines = f.readlines()
        for line in lines:
            i+=1
            if(i%100000==0):
                print(i)
            line= line.decode('utf-8','ignore').encode("utf-8")
            line_split = line.decode().split('\t\t')
            artist = line_split[0]
            movies = []
            if(len(line_split)>1):
                movies = set(map(remove_extra, line_split[1:]))
                movies =[m for m in movies if(len(m))>0]
            artist_list.append(artist)
            artist_movies_list.append(list(movies))     
    print(len(artist_list)," Lines found")
    artist_dataframe = pd.DataFrame(list(zip(artist_list, artist_movies_list)),columns=['director','movies'])
    return get_non_duplicated(artist_dataframe)

In [24]:
non_duplicated_director_df = get_director_data('data/director_movies.txt')

100000
200000
300000
389663  Lines found
Only 389610  unique names
389561  non repeated names
49  responsible for duplicates


In [25]:
non_duplicated_director_df

,director,movies
0,"&Oumlzkul, Ahmet Salih",[Ii (2013)]
1,"'Abd Al-Hamid, Ja'far","[Mesocaf (2011), Badgeless sur la Croisette (2..."
2,"'D.J'Arlia, Domenic",[Shell Never Know (2012)]
3,"'Dada' Pecori, Diego","[Makhno Beer (2010), Adam, Cantarella (2011)]"
4,"'Kid Niagara' Kallet, Harry",[Drug Demon Romance (2012)]
5,"'Kiki' Chansamone, S.",[]
6,"'Kusare, Mak (I)","[Comrade (2008), Baby Beautiful (2013/II)]"
7,"'Kusare, Mak (II)",[A Play Called a Temple Made of Clay (2014)]
8,"'Legend' Spivey, Larry",[The Crime City Diaries: Entry 1 - Crooked (20...
9,"'Noble Julz'Hamilton, Ulia",[Church Hurt (2015)]


In [30]:
movie_director_map = {}
for index, row in non_duplicated_director_df.iterrows():
    director = row['director']
    for i in row['movies']:
        movie_director_map[i] = director

In [68]:
director_ids = {}
x = 0
for i in movie_director_map.values():
    director_ids[i] = x
    x += 1

In [69]:
artist_ids = {}
x = 0
for i in movie_artist_map.values():
    artist_ids[i] = x
    x += 1

In [31]:
def get_non_duplicated(artist_dataframe):
    artist_counts = artist_dataframe[['movies','artist']].groupby('artist',as_index=False).agg(['count'])
    artist_counts.columns = [''.join(col).strip() for col in artist_counts.columns.values]
    artist_counts2 = artist_counts.reset_index()
    print('Only',artist_counts2['artist'].count(),' unique names')
    
    artist_counts2  = artist_counts2.loc[artist_counts2['moviescount'] >1]
    duplicate_artists = list(artist_counts2['artist'].values)
    #print(len(duplicate_artists),' repeated names')
    duplicated_artist_df  = artist_dataframe.loc[artist_dataframe['artist'].isin(duplicate_artists)]
    non_duplicated_artist_df  = artist_dataframe.loc[~artist_dataframe['artist'].isin(duplicate_artists)]
    print(non_duplicated_artist_df['movies'].count(),' non repeated names')
    fixed_duplicated_artist_df = duplicated_artist_df.groupby('artist').agg({'movies':'sum'})
    fixed_duplicated_artist_df['movies'] = fixed_duplicated_artist_df['movies'].apply(lambda x: list(set(x)))
    print(fixed_duplicated_artist_df['movies'].count(),' responsible for duplicates')
    non_duplicated_artist_df.append(fixed_duplicated_artist_df)
    return non_duplicated_artist_df

    

In [32]:
def get_artist_data(file_name):
    artist_list = []
    artist_movies_list = []
    i = 0
    with open(file_name,'rb') as f:
        lines = f.readlines()
        for line in lines:
            i+=1
            if(i%100000==0):
                print(i)
            line= line.decode('utf-8','ignore').encode("utf-8")
            line_split = line.decode().split('\t\t')
            artist = line_split[0]
            movies = []
            if(len(line_split)>1):
                movies = set(map(remove_extra, line_split[1:]))
                movies =[m for m in movies if(len(m))>0]
            artist_list.append(artist)
            artist_movies_list.append(list(movies))     
    print(len(artist_list)," Lines found")
    artist_dataframe = pd.DataFrame(list(zip(artist_list, artist_movies_list)),columns=['artist','movies'])
    return get_non_duplicated(artist_dataframe)
    

In [33]:
non_duplicated_actor_df = get_artist_data('data/actor_movies.txt')
non_duplicated_actress_df = get_artist_data('data/actress_movies.txt')

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2167653  Lines found
Only 2166817  unique names
2166116  non repeated names
701  responsible for duplicates
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1182813  Lines found
Only 1182404  unique names
1182084  non repeated names
320  responsible for duplicates


In [34]:
non_duplicated_artist_df = get_non_duplicated(non_duplicated_actor_df.append(non_duplicated_actress_df))
non_duplicated_artist_df['count'] = non_duplicated_artist_df['movies'].apply(lambda x: len(x))
non_duplicated_artist_df.head()

Only 3348112  unique names
3348024  non repeated names
88  responsible for duplicates


,artist,movies,count
0,"$, Claw",[],0
1,"$, Homo","[Suuri illusioni (1985), Nykytaiteen museo (19...",2
2,"$, Steve",[],0
3,"$hort, Too","[Beats, Rhymes & Life: The Travels of a Tribe ...",12
4,"$lim, Bee Moe","[Fatherhood 101 (2013), For Thy Love 2 (2009)]",2


In [118]:
from collections import defaultdict
movie_artist_map = defaultdict(list)

for index, row in non_duplicated_artist_df.iterrows():
    artist = row['artist']
    for i in row['movies']:
        movie_artist_map[i].append(artist)

In [119]:
print(len(movie_rating_map))
print(len(movie_director_map))
print(len(movie_artist_map))

348469
751566
643065


In [58]:

md_id = {}

directors = list()
for k, v in movie_director_map.items():
    directors.append(v)

In [38]:
available_movies = set(movie_artist_map.keys()).intersection(movie_rating_map.keys())

In [39]:
len(available_movies)

245508

In [40]:
sorted_movies = sorted(available_movies, key=lambda k : float(movie_rating_map[k]), reverse=True)[:250]

In [42]:
top_directors = set()
for movie in sorted_movies:
    if len(top_directors) >= 100:
        break;
    if movie in movie_director_map.keys():
        top_directors.add(movie_director_map[movie])

In [90]:
a2pr = pd.read_csv('a2pg.csv', names = ['artist', 'pagerank'])    
a2pr.head()

artist_pgr_map = {}

for i, r in a2pr.iterrows():
    artist_pgr_map[r['artist']] = float(r['pagerank'])

In [146]:
# artist_pgr_map
# actors = artist_pgr_map['Barclay, Jon Collin']
# print(actors)
# available_movies

In [139]:
def build_feature(movie):
    features = [0 for i in range(6)]
    actors = movie_artist_map[movie]
    page_ranks = []
    for actor in actors:
        try: 
            page_ranks.append(artist_pgr_map[actor])
        except:
            page_ranks.append(0.0)
    sorted_page_ranks = sorted(page_ranks, reverse=True)
    if(len(sorted_page_ranks)<5):
        for i in range(len(sorted_page_ranks)):
            features[i] = sorted_page_ranks[i]
    else:
        features[:5] = sorted_page_ranks[:5]

    if movie in movie_director_map.keys() and movie_director_map[movie] in top_directors:
        features[5] = 1
    return np.array(features)

feature_vector = np.vstack(map(build_feature, timer(available_movies)))






  0%|          | 0/245508 [00:00<?, ?it/s]




  1%|▏         | 3367/245508 [00:00<00:07, 33432.02it/s]




  3%|▎         | 6288/245508 [00:00<00:07, 31220.08it/s]




  4%|▍         | 9357/245508 [00:00<00:07, 31108.62it/s]




  5%|▌         | 12450/245508 [00:00<00:07, 31042.05it/s]




  6%|▋         | 15421/245508 [00:00<00:07, 30771.53it/s]




  8%|▊         | 18476/245508 [00:00<00:07, 30748.95it/s]




  9%|▉         | 21598/245508 [00:00<00:07, 30810.15it/s]




 10%|█         | 24691/245508 [00:00<00:07, 30826.80it/s]




 11%|█▏        | 27842/245508 [00:00<00:07, 30888.22it/s]




 13%|█▎        | 30819/245508 [00:01<00:07, 30388.36it/s]




 14%|█▎        | 33693/245508 [00:01<00:07, 30097.95it/s]




 15%|█▍        | 36700/245508 [00:01<00:06, 30134.46it/s]




 16%|█▋        | 40152/245508 [00:01<00:06, 30470.47it/s]




 18%|█▊        | 43364/245508 [00:01<00:06, 30504.35it/s]




 19%|█▉        | 46437/245508 [00:01<00:06, 30336.51it/s]




 20%|██        | 4943

In [140]:
target = []
for movie in timer(available_movies):
    target.append(float(movie_rating_map[movie]))
target_vector = np.array(target)






  0%|          | 0/245508 [00:00<?, ?it/s]




  0%|          | 413/245508 [00:00<00:59, 4086.08it/s]




  3%|▎         | 7730/245508 [00:00<00:06, 38523.24it/s]




  5%|▌         | 12472/245508 [00:00<00:05, 41388.84it/s]




 26%|██▌       | 63582/245508 [00:00<00:01, 158357.95it/s]




 59%|█████▉    | 144727/245508 [00:00<00:00, 288779.27it/s]




 97%|█████████▋| 237311/245508 [00:00<00:00, 394757.61it/s]




100%|██████████| 245508/245508 [00:00<00:00, 399816.89it/s]

In [142]:
from sklearn import linear_model

model = linear_model.LinearRegression()
model.fit(feature_vector, target_vector)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [153]:
test_movies = [
    "Batman v Superman: Dawn of Justice (2016)",
    "Mission: Impossible - Rogue Nation (2015)",
    "Minions (2015)"
]
test_feature_vector = np.vstack(map(build_feature, test_movies))

In [162]:
test_movies_prediction = model.predict(test_feature_vector)
print(test_movies_prediction)

[6.26022191 6.18882945 6.16806881]


In [164]:
prediction = model.predict(feature_vector)
prediction

array([6.28542989, 6.26729321, 6.30188938, ..., 6.28180152, 6.22874925,
       6.24026498])

In [165]:
import sklearn.metrics as mcs
mcs.mean_squared_error(target_vector, prediction)

1.7887928952677072

In [166]:
mcs.mean_absolute_error(target_vector, prediction)

1.0572257740094544

# Q13

In [186]:
actor_rating = {}
for i, row in non_duplicated_artist_df.iterrows():
    tmp = 0
    n = 0
    for m in row['movies']:
        if m in movie_rating_map.keys():
            tmp += movie_rating_map[m]
            n += 1
    if n == 0:
        actor_rating[row['artist']] = 0
    else:
        actor_rating[row['artist']] = (tmp*1.0)/n






233939it [00:29, 7950.76it/s] 

In [189]:
a = []
m = []
w = []

for i, row in non_duplicated_artist_df.iterrows():
    artist = row['artist']
    
    for k in row['movies']:
        a.append(artist)
        m.append(k)
        w.append(actor_rating[artist])

In [192]:
l = list(zip(a, m, w))
new_el = pd.DataFrame(l, columns=['artist', 'movie', 'weight'])

In [224]:
new_el.head()

,artist,movie,weight
0,"$, Homo",Suuri illusioni (1985),4.000000
1,"$, Homo",Nykytaiteen museo (1986),4.000000
2,"$hort, Too","Beats, Rhymes & Life: The Travels of a Tribe C...",6.511111
3,"$hort, Too",American Pimp (1999),6.511111
4,"$hort, Too",Porndogs: The Adventures of Sadie (2009),6.511111


In [229]:
movie_weights = defaultdict(list)
for i, row in timer(new_el.iterrows()):
    movie_weights[row['movie']].append(row['weight'])














0it [00:00, ?it/s]












1it [00:01,  1.18s/it]












752it [00:01, 578.13it/s]












2253it [00:01, 1608.31it/s]












3859it [00:01, 2571.05it/s]












5423it [00:01, 3387.60it/s]












6984it [00:01, 4106.16it/s]












8624it [00:01, 4788.76it/s]












10189it [00:01, 5359.22it/s]












11605it [00:02, 5790.47it/s]












13009it [00:02, 6175.83it/s]












14500it [00:02, 6571.64it/s]












16188it [00:02, 7018.37it/s]












17722it [00:02, 7363.62it/s]












19234it [00:02, 7602.05it/s]












20661it [00:02, 7853.19it/s]












22140it [00:02, 8106.79it/s]












23579it [00:02, 8320.75it/s]












25009it [00:02, 8508.78it/s]












26428it [00:03, 8695.44it/s]












27838it [00:03, 8857.51it/s]












29305it [00:03, 9036.59it/s]












30906it [00:03, 9238.32it/s]












32440it [00:03, 9422.00it/s]












33971it [00:03, 9588.37it/s

152922it [00:11, 13281.37it/s]












154427it [00:11, 13293.58it/s]












155922it [00:11, 13296.43it/s]












157376it [00:11, 13299.16it/s]












158803it [00:11, 13304.43it/s]












160454it [00:12, 13331.27it/s]












162101it [00:12, 13357.29it/s]












163753it [00:12, 13382.87it/s]












165362it [00:12, 13404.80it/s]












166974it [00:12, 13426.66it/s]












168565it [00:12, 13430.81it/s]












170097it [00:12, 13403.97it/s]












171499it [00:12, 13381.11it/s]












172813it [00:12, 13372.30it/s]












174164it [00:13, 13373.46it/s]












175624it [00:13, 13382.51it/s]












177087it [00:13, 13391.75it/s]












178572it [00:13, 13402.63it/s]












180026it [00:13, 13411.30it/s]












181546it [00:13, 13424.57it/s]












183045it [00:13, 13436.17it/s]












184513it [00:13, 13436.17it/s]












185944it [00:13, 13431.63it/s]












187407it [0

301924it [00:22, 13707.24it/s]












303606it [00:22, 13721.31it/s]












305191it [00:22, 13723.42it/s]












306725it [00:22, 13725.90it/s]












308264it [00:22, 13733.44it/s]












309777it [00:22, 13737.12it/s]












311273it [00:22, 13740.96it/s]












312757it [00:22, 13745.44it/s]












314296it [00:22, 13752.47it/s]












315795it [00:22, 13757.76it/s]












317293it [00:23, 13762.12it/s]












318882it [00:23, 13771.34it/s]












320514it [00:23, 13782.26it/s]












322202it [00:23, 13795.54it/s]












323854it [00:23, 13807.00it/s]












325465it [00:23, 13812.19it/s]












327042it [00:23, 13792.33it/s]












328465it [00:23, 13791.11it/s]












329866it [00:23, 13786.36it/s]












331230it [00:24, 13783.41it/s]












332577it [00:24, 13780.21it/s]












333912it [00:24, 13765.99it/s]












335188it [00:24, 13762.09it/s]












336734it [0

454348it [00:32, 14019.59it/s]












455914it [00:32, 14024.55it/s]












457472it [00:32, 14028.89it/s]












459025it [00:32, 14033.36it/s]












460600it [00:32, 14038.64it/s]












462196it [00:32, 14044.40it/s]












463766it [00:33, 14040.45it/s]












465254it [00:33, 14041.31it/s]












466724it [00:33, 14041.46it/s]












468177it [00:33, 14040.86it/s]












469610it [00:33, 14041.53it/s]












471216it [00:33, 14047.57it/s]












472699it [00:33, 14045.32it/s]












474140it [00:33, 14037.26it/s]












475575it [00:33, 14038.18it/s]












477174it [00:33, 14043.97it/s]












478717it [00:34, 14048.01it/s]












480194it [00:34, 14043.75it/s]












481613it [00:34, 14032.30it/s]












482938it [00:34, 14027.76it/s]












484239it [00:34, 14023.20it/s]












485528it [00:34, 14019.87it/s]












487098it [00:34, 14024.80it/s]












488515it [0

606600it [00:43, 14092.94it/s]












608065it [00:43, 14092.49it/s]












609509it [00:43, 14093.05it/s]












610985it [00:43, 14094.55it/s]












612446it [00:43, 14095.76it/s]












613904it [00:43, 14096.83it/s]












615359it [00:43, 14093.23it/s]












616759it [00:43, 14092.78it/s]












618155it [00:43, 14092.23it/s]












619569it [00:43, 14092.36it/s]












620969it [00:44, 14086.43it/s]












622306it [00:44, 14083.68it/s]












623694it [00:44, 14083.28it/s]












625170it [00:44, 14084.79it/s]












626690it [00:44, 14087.29it/s]












628132it [00:44, 14088.01it/s]












629614it [00:44, 14089.67it/s]












631059it [00:44, 14090.06it/s]












632498it [00:44, 14090.60it/s]












633966it [00:44, 14091.79it/s]












635433it [00:45, 14093.06it/s]












636897it [00:45, 14094.31it/s]












638379it [00:45, 14095.93it/s]












639843it [0

...

6061154it [06:46, 14918.77it/s]












6062756it [06:46, 14919.03it/s]












6064399it [06:46, 14919.40it/s]












6066117it [06:46, 14919.96it/s]












6067762it [06:46, 14919.94it/s]












6069419it [06:46, 14920.35it/s]












6071117it [06:46, 14920.86it/s]












6072772it [06:46, 14921.25it/s]












6074451it [06:47, 14921.70it/s]












6076114it [06:47, 14922.13it/s]












6077788it [06:47, 14922.57it/s]












6079451it [06:47, 14922.86it/s]












6081111it [06:47, 14923.28it/s]












6082761it [06:47, 14923.53it/s]












6084438it [06:47, 14923.73it/s]












6086054it [06:47, 14923.98it/s]












6087661it [06:47, 14924.02it/s]












6089281it [06:48, 14924.34it/s]












6090929it [06:48, 14924.72it/s]












6092536it [06:48, 14924.91it/s]












6094133it [06:48, 14925.05it/s]












6095715it [06:48, 14925.23it/s]












6097353it [06:48, 14925.58it/s]


6222662it [06:56, 14948.92it/s]












6224319it [06:56, 14949.02it/s]












6225972it [06:56, 14949.41it/s]












6227658it [06:56, 14949.87it/s]












6229305it [06:56, 14949.83it/s]












6230962it [06:56, 14950.21it/s]












6232577it [06:56, 14950.31it/s]












6234168it [06:56, 14950.28it/s]












6235738it [06:57, 14950.46it/s]












6237399it [06:57, 14950.86it/s]












6239051it [06:57, 14951.23it/s]












6240661it [06:57, 14951.27it/s]












6242295it [06:57, 14951.60it/s]












6243980it [06:57, 14952.05it/s]












6245640it [06:57, 14952.45it/s]












6247338it [06:57, 14952.93it/s]












6248991it [06:57, 14953.15it/s]












6250624it [06:58, 14953.36it/s]












6252292it [06:58, 14953.77it/s]












6253978it [06:58, 14954.22it/s]












6255626it [06:58, 14954.41it/s]












6257250it [06:58, 14954.30it/s]












6258941it [06:58, 14954.77it/s]


6384018it [07:06, 14976.13it/s]












6385668it [07:06, 14976.48it/s]












6387349it [07:06, 14976.92it/s]












6389064it [07:06, 14977.42it/s]












6390731it [07:06, 14977.75it/s]












6392389it [07:06, 14978.08it/s]












6394041it [07:06, 14978.18it/s]












6395702it [07:06, 14978.56it/s]












6397333it [07:07, 14978.78it/s]












6398951it [07:07, 14979.03it/s]












6400639it [07:07, 14979.47it/s]












6402305it [07:07, 14979.77it/s]












6403950it [07:07, 14980.05it/s]












6405627it [07:07, 14980.46it/s]












6407267it [07:07, 14980.66it/s]












6408938it [07:07, 14981.06it/s]












6410574it [07:07, 14981.34it/s]












6412204it [07:08, 14981.36it/s]












6413831it [07:08, 14981.66it/s]












6415499it [07:08, 14982.05it/s]












6417121it [07:08, 14982.28it/s]












6418795it [07:08, 14982.69it/s]












6420427it [07:08, 14982.92it/s]


6546754it [07:16, 15007.12it/s]












6548411it [07:16, 15007.46it/s]












6550067it [07:16, 15007.54it/s]












6551686it [07:16, 15007.79it/s]












6553303it [07:16, 15007.46it/s]












6554899it [07:16, 15007.68it/s]












6556570it [07:16, 15008.07it/s]












6558249it [07:16, 15008.47it/s]












6559868it [07:17, 15008.51it/s]












6561547it [07:17, 15008.92it/s]












6563227it [07:17, 15009.33it/s]












6564901it [07:17, 15009.72it/s]












6566569it [07:17, 15010.10it/s]












6568222it [07:17, 15010.26it/s]












6569922it [07:17, 15010.71it/s]












6571592it [07:17, 15011.10it/s]












6573247it [07:17, 15011.21it/s]












6574874it [07:17, 15011.37it/s]












6576483it [07:18, 15011.49it/s]












6578144it [07:18, 15011.85it/s]












6579755it [07:18, 15011.90it/s]












6581403it [07:18, 15012.24it/s]












6583070it [07:18, 15012.62it/s]


6708597it [07:26, 15031.95it/s]












6710224it [07:26, 15032.21it/s]












6711848it [07:26, 15032.20it/s]












6713517it [07:26, 15032.57it/s]












6715161it [07:26, 15032.88it/s]












6716782it [07:26, 15033.05it/s]












6718390it [07:26, 15032.85it/s]












6719951it [07:27, 15032.98it/s]












6721507it [07:27, 15033.04it/s]












6723119it [07:27, 15033.27it/s]












6724724it [07:27, 15033.50it/s]












6726302it [07:27, 15033.61it/s]












6727872it [07:27, 15033.63it/s]












6729495it [07:27, 15033.88it/s]












6731068it [07:27, 15034.02it/s]












6732750it [07:27, 15034.42it/s]












6734354it [07:27, 15034.56it/s]












6735948it [07:28, 15034.48it/s]












6737574it [07:28, 15034.75it/s]












6739150it [07:28, 15034.70it/s]












6740699it [07:28, 15034.77it/s]












6742242it [07:28, 15034.75it/s]












6743773it [07:28, 15034.34it/s]


In [303]:
def build_feature_q13(movie):
#     print(movie)
    features = [0]
#     actors = movie_artist_map[movie]
#     if movie in movie_director_map.keys() and movie_director_map[movie] in top_directors:
#         features[1] = 1

#     ratings = list(new_el[new_el['movie']==movie]['weight'])
#     if len(ratings) < 6:
#         for i in range(len(ratings)):
#             features[i] = ratings[i]
#     else:
#         features[:6] = ratings[:6]
    features[0] = np.mean(movie_weights[i])
    return np.array(features)


fv = list()

for i in timer(available_movies):
#     features = [0] * 6
    
#     ratings = movie_weights[i]
#     if len(ratings) > 0:
#         ratings.sort(reverse=True)
    
#     if len(ratings) < 5:
#         for i in range(len(ratings)):
#             features[i] = ratings[i]
#     else:
#         features[:5] = ratings[:5]
    features = [0] * 4
    
    ratings = movie_weights[i]
    features[0] = np.mean(ratings)
    features[1] = max(ratings)
    features[2] = min(ratings)
    
#     features[0] = ratings
    if i in movie_director_map.keys() and movie_director_map[i] in top_directors:
        features[3] = 1
    
    fv.append(features)


feature_vector_13 = np.array(fv)
# feature_vector_13 = np.vstack(map(build_feature_q13, timer(available_movies)))














  0%|          | 0/245508 [00:00<?, ?it/s]












  1%|          | 2318/245508 [00:00<00:10, 22764.55it/s]












  2%|▏         | 4791/245508 [00:00<00:10, 23621.01it/s]












  3%|▎         | 6396/245508 [00:00<00:11, 20808.95it/s]












  3%|▎         | 7729/245508 [00:00<00:13, 17435.77it/s]












  4%|▍         | 10480/245508 [00:00<00:12, 19163.47it/s]












  5%|▌         | 13222/245508 [00:00<00:11, 20525.73it/s]












  7%|▋         | 16717/245508 [00:00<00:10, 22490.94it/s]












  9%|▊         | 20961/245508 [00:00<00:09, 24776.89it/s]












 10%|█         | 25755/245508 [00:00<00:08, 27250.83it/s]












 12%|█▏        | 30323/245508 [00:01<00:07, 29062.73it/s]












 14%|█▍        | 34161/245508 [00:01<00:07, 29704.80it/s]












 16%|█▌        | 38176/245508 [00:01<00:06, 30529.03it/s]












 17%|█▋        | 42141/245508 [00:01<00:06, 31226.54it/s]












 19%|█▊        | 46021/245

In [304]:
# feature_vector_13 = np.array(fv)
print(feature_vector_13)

[[6.4617823  8.         5.24       0.        ]
 [6.73474691 7.725      5.14473684 0.        ]
 [4.77298012 6.78846154 3.7        0.        ]
 ...
 [6.26316262 6.916      5.87058824 0.        ]
 [6.98441416 8.65       5.9        0.        ]
 [6.7354749  7.7        5.27920792 0.        ]]


In [305]:
target_13 = []
for movie in timer(available_movies):
    target_13.append(float(movie_rating_map[movie]))
target_vector_13 = np.array(target)














  0%|          | 0/245508 [00:00<?, ?it/s]












 30%|██▉       | 72542/245508 [00:00<00:00, 722091.12it/s]












 63%|██████▎   | 155189/245508 [00:00<00:00, 773589.54it/s]












100%|██████████| 245508/245508 [00:00<00:00, 824799.02it/s]

In [306]:
model_13 = linear_model.LinearRegression()
model_13.fit(feature_vector_13, target_vector_13)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [307]:
test_movies_13 = [
    "Batman v Superman: Dawn of Justice (2016)",
    "Mission: Impossible - Rogue Nation (2015)",
    "Minions (2015)"]

fv_test = list()

for i in timer(test_movies_13):
#     features = [0] * 6
    
#     ratings = movie_weights[i]
    
#     if len(ratings) > 0:
#         ratings.sort(reverse=True)
    
#     if len(ratings) < 5:
#         for i in range(len(ratings)):
#             features[i] = ratings[i]
#     else:
#         features[:5] = ratings[:5]
    
# #     features[0] = ratings
#     if i in movie_director_map.keys() and movie_director_map[i] in top_directors:
#         features[5] = 1
    features = [0] * 4
    
    ratings = movie_weights[i]
    features[0] = np.mean(ratings)
    features[1] = max(ratings)
    features[2] = min(ratings)
    
    fv_test.append(features)
    
print(fv_test)
test_feature_vector_13 = np.array(fv_test)
# test_feature_vector_13 = np.vstack(map(build_feature_q13, test_movies_13))
print(test_feature_vector_13)














  0%|          | 0/3 [00:00<?, ?it/s]












100%|██████████| 3/3 [00:00<00:00, 1268.06it/s]

[[4.931926845159866, 9.35, 0.0, 0], [5.427342491453443, 9.1, 0.0, 0], [6.600309245202448, 7.708333333333332, 0.0, 0]]
[[4.93192685 9.35       0.         0.        ]
 [5.42734249 9.1        0.         0.        ]
 [6.60030925 7.70833333 0.         0.        ]]


In [308]:
test_movies_prediction_13 = model_13.predict(test_feature_vector_13)
print(test_movies_prediction_13)

[4.55826703 4.88113766 5.35168548]


In [309]:
prediction_13 = model_13.predict(feature_vector_13)
prediction_13

array([6.77611728, 6.87751862, 4.4958962 , ..., 6.38396892, 7.63302379,
       6.90572095])

In [310]:
mcs.mean_squared_error(target_vector_13, prediction_13)

0.4943748601415893

In [311]:
mcs.mean_absolute_error(target_vector_13, prediction_13)

0.5274777374190518